In [ ]:
import pandas as pd
import numpy as np
import pickle
import tkinter as tk
from tkinter import messagebox, filedialog

# ------------------------------
# Load Models
# ------------------------------
try:
    with open("XGBoost_WOA_Sorptivity.pkl", "rb") as file:
        model_sorp = pickle.load(file)
except Exception as e:
    messagebox.showerror("Model Load Error", f"Failed to load Sorptivity model: {e}")

try:
    with open("XGBoost_WOA_RCPT.pkl", "rb") as file:
        model_rcpt = pickle.load(file)
except Exception as e:
    messagebox.showerror("Model Load Error", f"Failed to load RCPT model: {e}")

# ------------------------------
# GUI Setup
# ------------------------------
root = tk.Tk()
root.title("Concrete Properties Prediction using Hybrid XGBoost-WOA")
root.geometry("780x800")
root.configure(bg="white")

canvas = tk.Canvas(root, width=760, height=780, bg='skyblue')
canvas.pack(pady=10)

# Title
canvas.create_text(380, 25, text="Prediction of Sorptivity and RCPT", font=('Times New Roman', 16, 'bold'), fill='black')

# Developer Info
devs = [
    "Developed by:",
    "1. Dr. Divesh Ranjan Kumar, Chulalongkorn University",
    "2. Dr. Shashikant Kumar, Muzaffarpur Institute of Technology",
    "3. Dr. Teerapong Senjuntichai, Chulalongkorn University",
    "4. Dr. Sakdirat Kaewunruen, University of Birmingham"
]

y = 50
for dev in devs:
    canvas.create_text(20, y, anchor="w", text=dev, font=('Times New Roman', 11), fill='black')
    y += 20

canvas.create_text(20, y, anchor="w", text="Contact: ranjandivesh453@gmail.com", font=('Times New Roman', 11), fill='black')
y += 30

canvas.create_text(20, y, anchor="w", text="Input Parameters", font=('Times New Roman', 12, 'bold', 'underline'), fill='black')
y += 30

# ------------------------------
# Input Fields
# ------------------------------
labels = [
    'OPC (kg)', 'FA (kg)', 'SF (kg)',
    'Coarse Aggregate  (kg)', 'Fine Aggregate  (kg)',
    'Water (kg)', 'T-500 (sec)', 'Maximum Spread Diameter (mm)',
    'J-ring test (mm)', 'L-box blocking ratio (h₂/h₁)',
    'V-funnel time (sec)', 'Temperature (°C)'
]
entries = []

for label_text in labels:
    label = tk.Label(root, text=label_text, font=('Times New Roman', 10), bg='white')
    canvas.create_window(20, y, anchor="w", window=label)

    entry = tk.Entry(root)
    canvas.create_window(400, y, window=entry)
    entries.append(entry)
    y += 30

# ------------------------------
# Output Section
# ------------------------------
canvas.create_text(20, y + 10, anchor="w", text="Output", font=('Times New Roman', 12, 'bold', 'underline'), fill='black')
y += 50

sorp_label = tk.Label(root, text="Sorptivity (×10⁻⁴ mm/√s):", font=('Times New Roman', 11), bg='white')
canvas.create_window(20, y, anchor="w", window=sorp_label)
sorp_output = tk.Label(root, text="", font=('Times New Roman', 11), bg='white', relief='solid', width=20)
canvas.create_window(400, y, anchor="w", window=sorp_output)
y += 40

rcpt_label = tk.Label(root, text="RCPT (Coulomb):", font=('Times New Roman', 11), bg='white')
canvas.create_window(20, y, anchor="w", window=rcpt_label)
rcpt_output = tk.Label(root, text="", font=('Times New Roman', 11), bg='white', relief='solid', width=20)
canvas.create_window(400, y, anchor="w", window=rcpt_output)
y += 50

# ------------------------------
# Functions
# ------------------------------
def predict():
    try:
        input_values = [float(entry.get()) for entry in entries]
    except ValueError:
        messagebox.showerror("Invalid Input", "Please enter valid numeric values.")
        return

    df_input = pd.DataFrame([input_values], columns=labels)
    try:
        pred_sorp = round(model_sorp.predict(df_input)[0], 2)
        pred_rcpt = round(model_rcpt.predict(df_input)[0], 2)
        sorp_output.config(text=str(pred_sorp))
        rcpt_output.config(text=str(pred_rcpt))
    except Exception as e:
        messagebox.showerror("Prediction Error", str(e))


def clear_inputs():
    for entry in entries:
        entry.delete(0, tk.END)
    sorp_output.config(text="")
    rcpt_output.config(text="")


def fill_sample():
    sample = [300, 100, 10, 1000, 850, 180, 4, 700, 12, 0.85, 9, 30]  # Example sample values
    for entry, val in zip(entries, sample):
        entry.delete(0, tk.END)
        entry.insert(0, str(val))


def export_to_csv():
    input_values = [entry.get() for entry in entries]
    if not all(input_values):
        messagebox.showwarning("Incomplete Input", "Fill in all fields before exporting.")
        return
    try:
        df = pd.DataFrame([input_values], columns=labels)
        df['Predicted Sorptivity'] = sorp_output.cget("text")
        df['Predicted RCPT'] = rcpt_output.cget("text")

        file_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV Files", "*.csv")])
        if file_path:
            df.to_csv(file_path, index=False)
            messagebox.showinfo("Success", "File saved successfully.")
    except Exception as e:
        messagebox.showerror("Export Error", str(e))

# ------------------------------
# Buttons
# ------------------------------
predict_btn = tk.Button(root, text="Predict", command=predict, bg='green', fg='white', font=('Times New Roman', 11, 'bold'))
canvas.create_window(100, y, window=predict_btn)

clear_btn = tk.Button(root, text="Clear", command=clear_inputs, bg='red', fg='white', font=('Times New Roman', 11, 'bold'))
canvas.create_window(200, y, window=clear_btn)

sample_btn = tk.Button(root, text="Fill Sample", command=fill_sample, bg='orange', font=('Times New Roman', 11))
canvas.create_window(320, y, window=sample_btn)

export_btn = tk.Button(root, text="Export CSV", command=export_to_csv, bg='blue', fg='white', font=('Times New Roman', 11))
canvas.create_window(440, y, window=export_btn)

root.mainloop()
